In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Conv2D, MaxPooling3D, Conv3D, MaxPooling2D, TimeDistributed,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB0

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


import matplotlib.pyplot as plt

import numpy as np
import tensorflow_addons as tfa

In [ ]:

img_size = (150,150)
X = np.load("../Data/COVID CXR/covid_org6000_150_X.npy")
Y = np.load("../Data/COVID CXR/covid_org6000_150_Y.npy")

In [ ]:

index = [i for i in range(X.shape[0])]
np.random.shuffle(index) # 打乱索引


X2 = X[index]/255.0
Y2 = Y[index]

In [ ]:
Y3 = tf.keras.utils.to_categorical(Y2, 2)

In [ ]:
def build_model():
    inputs = layers.Input(shape=(150, 150, 3))
    #model = ResNet50(include_top=False, input_tensor=inputs, weights = None)
    model = DenseNet121(include_top=False, input_tensor=inputs, weights = None)
    #model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=None)



    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(2, activation="softmax", name="pred")(x)

    # Compile

    model = tf.keras.Model(inputs, outputs, name="DenseNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=10)
earlystop = EarlyStopping(monitor='val_accuracy', patience=15)

In [ ]:
seed = 101
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvacc = []
cvauc = []
i = 1

for train, test in kfold.split(X2,Y2):

    model = build_model()
    # checkpoint
    filepath = "../Data/models/150_cnn{0}_COVID19_densenet121.h5".format(str(i))
    if os.path.exists(filepath):
        os.remove(filepath)

    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint,earlystop]

    model.fit(X2[train], Y3[train],validation_split=0.2, epochs=300, batch_size=32, verbose=1,shuffle=True,callbacks = callbacks_list)
    # evaluate the model
    model.load_weights(filepath)
    acc = model.evaluate(X2[test], Y3[test], verbose=1)
    y_pred = model.predict(X2[test])
    with open('../Data/models/150_{0}_COVID19_cnn_densenet121.csv'.format(str(i)),'w') as file:
        file.write('true_label,predict_label'+'\n')
        for a,b in zip(Y2[test],y_pred):
            file.write(str(a) + ','+str(np.argmax(b))+'\n')
    auc = roc_auc_score(Y3[test],y_pred)
    
    print("%s: %.2f%%" % (model.metrics_names[1], acc[1]*100))
    print("auc: %.2f"%(auc))
    cvacc.append(acc[1] * 100)
    cvauc.append(auc)
    i+=1
print("mean acc %.2f%% (+/- %.2f%%)" % (np.mean(cvacc), np.std(cvacc)))
print("mean auc %.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))

In [ ]:
cvacc

In [ ]:
np.mean(cvacc)

In [ ]:
cvauc

In [ ]:
np.mean(cvauc)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

In [ ]:
def calculate_metric(gt, pred): 
    pred2 = []
    for i in pred:
        if i>0.5:
            pred2.append(1)
        else:
            pred2.append(0)
    confusion = confusion_matrix(gt,pred2)
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    acc = (TP+TN)/float(TP+TN+FP+FN)
    auc = roc_auc_score(gt,pred)
    precision = TP/float(TP+FP)
    f1_score_ = f1_score(gt,pred2)
    sensitivity = TP / float(TP+FN)
    specificity = TN / float(TN+FP)
    
    return round(acc,4),round(auc,4),round(f1_score_,4), round(precision,4),round(sensitivity,4), round(specificity,4)

In [ ]:
df1 = pd.read_csv('../Data/models/150_1_COVID19_cnn_densenet121.csv')


calculate_metric(df1['true_label'],df1['predict_label'])

In [ ]:
df2 = pd.read_csv('../Data/models/150_2_COVID19_cnn_densenet121.csv')


calculate_metric(df2['true_label'],df2['predict_label'])

In [ ]:
df3 = pd.read_csv('../Data/models/150_3_COVID19_cnn_densenet121.csv')


calculate_metric(df3['true_label'],df3['predict_label'])

In [ ]:
df4 = pd.read_csv('../Data/models/150_4_COVID19_cnn_densenet121.csv')

calculate_metric(df4['true_label'],df4['predict_label'])

In [ ]:
df5 = pd.read_csv('../Data/models/150_5_COVID19_cnn_densenet121.csv')


calculate_metric(df5['true_label'],df5['predict_label'])